In [1]:
!pip install newspaper3k
!pip install kafka-python

done
  Stored in directory: /home/jovyan/.cache/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c35da20627c013f438ed34
  Stored in directory: /home/jovyan/.cache/pip/wheels/81/2b/43/a02ede72324dd40cdd7ca53aad718c7710628e91b8b0dc0f02
  Stored in directory: /home/jovyan/.cache/pip/wheels/8c/69/b7/f52763c41c5471df57703a0ef718a32a5e81ee35dcf6d4f97f
  Stored in directory: /home/jovyan/.cache/pip/wheels/97/8a/10/d646015f33c525688e91986c4544c68019b19a473cb33d3b55
Successfully built feedfinder2 jieba3k tinysegmenter feedparser nltk
     |████████████████████████████████| 266kB 3.1MB/s 


In [6]:
import newspaper

from kafka import KafkaProducer
from kafka.errors import KafkaError
from kafka import KafkaConsumer

from json import loads, dumps
import time

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
producer = KafkaProducer(
    bootstrap_servers=['3.18.83.70:9092'],
    value_serializer=lambda x: dumps(x).encode('utf-8')
)

In [4]:
urls = ['http://cnn.com', \
        'http://nytimes.com', \
        'https://www.theguardian.com/us', \
        'https://news.google.com/', \
        'https://medium.com/', \
        'https://www.usatoday.com/']

In [ ]:
while True:
    articles = []
    for url in urls:
        articles.append(newspaper.build(url).articles)
    
    for i in range(len(articles)):
        for art in articles[i]:
            try:
                art.download()
                art.parse()

                current_news = {}
                current_news['source'] = urls[i]
                current_news['title'] = art.title
                current_news['authors'] = art.authors
                current_news['content'] = art.text
                current_news['publish_date'] = str(art.publish_date)
                current_news['link'] = art.url

                art.nlp()
                current_news['summary'] = art.summary
                current_news['keywords'] = art.keywords

                print("Send start, the news title is {}".format(current_news['title']))
                producer.send('testNews', value=current_news)
            except Exception as e:
                print(e)
        
    time.sleep(30)

Send start, the news title is How to Train Your Dragon: The Hidden World
Send start, the news title is The LEGO Movie:The Second Part
Send start, the news title is A Game of Thrones: A Song of Ice and Fire: Book One by George R. R. Martin
Send start, the news title is The Art Of War by Sun Tzu
Send start, the news title is The Mueller Report by The Washington Post
Send start, the news title is The Subtle Art of Not Giving a F*ck: A Counterintuitive Approach to Living a Good Life by Mark Manson
Send start, the news title is Where the Crawdads Sing by Delia Owens
Send start, the news title is Unfu*k Yourself: Get Out of Your Head and into Your Life by Gary John Bishop
Send start, the news title is A Game of Thrones: A Song of Ice and Fire: Book One by George R. R. Martin
Send start, the news title is A Clash of Kings: A Song of Ice and Fire: Book Two by George R. R. Martin
Send start, the news title is The Trouble With Vampires: An Argeneau Novel by Lynsay Sands
Send start, the news titl

In [ ]:
producer.close()